In [1]:
from aocd import submit
import numpy as np
import re
from itertools import product

In [2]:
YYYY=2021
DD=22

In [3]:
def translate(s):
    parse=re.compile("[-+]?[0-9]+")
    s1 = 1 if  s[0:2] == "on" else 0
    s2 = [int(n) for n in parse.findall(s)]
    s2 = [(s2[0], s2[1]), (s2[2], s2[3]), (s2[4], s2[5])]

    return [s1, tuple(s2)]
    
def load(filename):
    with open(filename,'r') as f:
         lines = [translate(s)  for s in f]
    return lines
test1 = load("test1.dat")
test2 = load("test2.dat")
test3 = load("test3.dat")
test2

[[1, ((-20, 26), (-36, 17), (-47, 7))],
 [1, ((-20, 33), (-21, 23), (-26, 28))],
 [1, ((-22, 28), (-29, 23), (-38, 16))],
 [1, ((-46, 7), (-6, 46), (-50, -1))],
 [1, ((-49, 1), (-3, 46), (-24, 28))],
 [1, ((2, 47), (-22, 22), (-23, 27))],
 [1, ((-27, 23), (-28, 26), (-21, 29))],
 [1, ((-39, 5), (-6, 47), (-3, 44))],
 [1, ((-30, 21), (-8, 43), (-13, 34))],
 [1, ((-22, 26), (-27, 20), (-29, 19))],
 [0, ((-48, -32), (26, 41), (-47, -37))],
 [1, ((-12, 35), (6, 50), (-50, -2))],
 [0, ((-48, -32), (-32, -16), (-15, -5))],
 [1, ((-18, 26), (-33, 15), (-7, 46))],
 [0, ((-40, -22), (-38, -28), (23, 41))],
 [1, ((-16, 35), (-41, 10), (-47, 6))],
 [0, ((-32, -23), (11, 30), (-14, 3))],
 [1, ((-49, -5), (-3, 45), (-29, 18))],
 [0, ((18, 30), (-20, -8), (-3, 13))],
 [1, ((-41, 9), (-7, 43), (-33, 15))],
 [1, ((-54112, -39298), (-85059, -49293), (-27449, 7877))],
 [1, ((967, 23432), (45373, 81175), (27513, 53682))]]

In [4]:
data_in = load("input.dat")

# Teil a

In [ ]:
"""
0..100
-50 = 0
50 = 100
"""
a= 'on x=10..12,y=10..12,z=10..12'
parse=re.compile("\d+")
parse.findall(a)

In [ ]:
def transform(cubes):
    c=list(cubes)
    for i,v in enumerate(cubes):
        c[i] = 50+(min(50, v) if i % 2 else max(-50,v))
    return c

In [ ]:
transform(test1[0][1])

In [ ]:
def solution_a(data):
    c = np.zeros((101,101,101), int)
    for s, d in data:
        x1,x2, y1,y2, z1, z2 = transform(d)
        c[x1:x2+1,y1:y2+1,z1:z2+1] = s
        #print(c.sum())
    return c.sum()

In [ ]:
solution_a(test2)

In [ ]:
submit(solution_a(data_in),part="a",day=DD, year=YYYY)

# Teil b

In [16]:
def isin(a, b):
    # is a in b?
    if a == "v":
        return False
    a1,a2=a
    b1,b2=b
    return (b1 <= a1 and a2 <= b2)    


def create_interval(l,r):
    if r-l > 1:
        return set([(l,l), (l+1,r-1), (r,r)])
    return set([(l,l), (r,r)])
    
def list2interval(l):
    result = set()
    for l,r in zip(l[:-1],l[1:]):
        result |= create_interval(l,r)
    return list(result)
    
            
def init(data):
    X,Y,Z = set(),set(),set()
    for switch, cube in data:
        x,y,z = cube
        X|=set([x[0],x[1]])
        Y|=set([y[0],y[1]])
        Z|=set([z[0],z[1]])
    X=list2interval(sorted(list(X)))
    #print(len(X), flush=True)
    Y=list2interval(sorted(list(Y)))
    #print(len(Y), flush=True)
    Z=list2interval(sorted(list(Z)))
    #print(len(Z), flush=True)
    return sorted(X),sorted(Y),sorted(Z)


def split_cube2(cube, X,Y,Z):
    x,y,z = cube
    _x=[xx for i, xx in  enumerate(X) if isin(xx, x)]
    _y=[yy for i, yy in  enumerate(Y) if isin(yy, y)]
    _z=[zz for i, zz in  enumerate(Z) if isin(zz, z)]
    return _x, _y, _z

def get_cube(cube, X,Y,Z):
    x,y,z = cube
    _x=[i for i, xx in  enumerate(X) if isin(xx, x)]
    _y=[i for i, yy in  enumerate(Y) if isin(yy, y)]
    _z=[i for i, zz in  enumerate(Z) if isin(zz, z)]
    return ((min(_x), max(_x)),(min(_y), max(_y)),(min(_z), max(_z)))

    
def count(meta, x,y,z):
    s = 0
    dimx,dimy,dimz=meta.shape
    
    for xx in range(dimx):
        print(xx)
        xxx = (1+x[xx][1]-x[xx][0])
        for yy in range(dimy):
            yyy = (1+y[yy][1]-y[yy][0])
            for zz in range(dimz):
                if meta[xx,yy,zz]:
                    s += xxx*yyy*(1+z[zz][1]-z[zz][0])
    return s
                

def solution_b(data):
    x,y,z = init(data)
    dimx, dimy, dimz = len(x), len(y), len(z)
    meta = np.zeros((dimx, dimy, dimz), int)
    #return meta.shape
    x,y,z = init(data)
    for i, (switch, cube) in enumerate(data):
        print(i,cube)
        (x1,x2),(y1,y2),(z1,z2) = get_cube(cube, x,y,z)
        #print((x1,x2),(y1,y2),(z1,z2))
        meta[x1:x2+1,y1:y2+1,z1:z2+1] = switch # *(1+x2-x1)*(1+y2-y1)*(1+z2-z1)
        #print(meta)
    return count(meta,x,y,z)
                    

In [17]:
solution_b(test1)

0 ((10, 12), (10, 12), (10, 12))
1 ((11, 13), (11, 13), (11, 13))
2 ((9, 11), (9, 11), (9, 11))
3 ((10, 10), (10, 10), (10, 10))
0
1
2
3
4


39

In [11]:
solution_b(data_in)

0 ((-11, 33), (-6, 40), (-16, 37))
1 ((-44, 10), (-24, 30), (-24, 22))
2 ((-34, 15), (-21, 27), (-33, 11))
3 ((-42, 12), (-43, 9), (1, 48))
4 ((-31, 21), (-11, 42), (-4, 49))
5 ((0, 44), (-13, 37), (-30, 14))
6 ((-41, 12), (-32, 17), (-7, 43))
7 ((-21, 27), (-16, 30), (-33, 15))
8 ((-28, 24), (-12, 42), (-6, 45))
9 ((-15, 30), (-32, 14), (-48, 5))
10 ((-32, -17), (-6, 11), (-16, -5))
11 ((-29, 17), (-6, 48), (-27, 17))
12 ((-34, -20), (-22, -6), (-24, -11))
13 ((-39, 5), (-24, 22), (-49, 2))
14 ((-48, -32), (32, 45), (31, 44))
15 ((-40, 5), (-9, 39), (-2, 43))
16 ((15, 34), (3, 19), (-46, -30))
17 ((-10, 36), (-8, 43), (-36, 8))
18 ((8, 22), (2, 11), (27, 44))
19 ((-26, 22), (-17, 31), (-23, 30))
20 ((55966, 74708), (9028, 39775), (-23782, -12392))
21 ((46912, 62342), (34758, 55202), (5912, 31510))
22 ((63138, 67807), (40967, 45502), (-23583, -10004))
23 ((-73495, -56406), (41511, 71705), (5127, 32006))
24 ((-23106, -1748), (-86112, -68506), (-39397, -15794))
25 ((-52779, -45652), (190

209 ((26397, 48116), (49913, 64456), (32763, 46893))
210 ((-40166, -6453), (54294, 88546), (19469, 46914))
211 ((-11587, -3465), (-76385, -45142), (-55088, -47775))
212 ((25337, 58818), (-73241, -50645), (-60403, -31242))
213 ((18670, 27833), (10699, 25997), (-78411, -53777))
214 ((57409, 75450), (40962, 60348), (11226, 25913))
215 ((55748, 74851), (-54477, -43708), (-28494, -15041))
216 ((26830, 53176), (-67735, -51525), (13985, 45082))
217 ((-8911, 13239), (-74882, -67577), (-51935, -26067))
218 ((63953, 87466), (35424, 47855), (-20625, -3868))
219 ((-20694, 2154), (-3805, 15541), (68000, 85521))
220 ((-64858, -38773), (-22979, -12516), (-72865, -52375))
221 ((-36214, -19494), (-48856, -31044), (55849, 67059))
222 ((18769, 39214), (59223, 74065), (-25526, -7466))
223 ((4093, 19302), (65172, 80228), (-18875, 4459))
224 ((-72853, -64913), (-44437, -23173), (16546, 27389))
225 ((-74000, -63343), (-17952, -11976), (26071, 46600))
226 ((55718, 89196), (8034, 31868), (23749, 41136))
227 ((

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1263946820845866